In [1]:
import tensorflow as tf
tf.python.control_flow_ops = tf

import os
import pandas as pd
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, \
MaxPooling2D, Conv2D, Lambda, Cropping2D, Convolution2D,\
AveragePooling2D
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.cross_validation import train_test_split

from copy import deepcopy
from sklearn.utils import shuffle


from generator import generator
from get_images import get_images

from keras.optimizers import Adam


Using TensorFlow backend.
/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
datadirs=['/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_forward/',
                 '/notebooks/udacity/new_training/map2_backward/',
                 '/notebooks/udacity/new_training/map2_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_recovery_backward/',
                   '/notebooks/udacity/new_training/map1_error_correction/',
                   '/notebooks/udacity/new_training/map2_error_correction/'
         ]

#datadirs=['/notebooks/udacity/car_data/data/']

images=get_images(datadirs,0.1)

In [3]:
#different preprocessing methods
size=(40,80)

def preprocessing_resize(im):
    return cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2YUV),(size[1],size[0]))

def preprocessing_yuv(im):
    return cv2.cvtColor(im, cv2.COLOR_BGR2YUV)

def proc_img(img): # input is 160x320x3
    img = img[59:138:2, 0:-1:2, :] # select vertical region and take each second pixel to reduce image dimensions
    img = (img / 127.5) - 1.0 # normalize colors from 0-255 to -1.0 to 1.0
    return img # return 40x160x3 image




In [4]:
from car_models import *

In [5]:
from generator import generator

In [6]:
names, steering, inverse = images.img.values, images.real.values, images.inverse.values

In [7]:
from sklearn.cross_validation import train_test_split 

names_train, names_valid, steering_train, steering_valid, inverse_train, inverse_valid = \
    train_test_split(names, steering, inverse, test_size=0.02)
    
names_valid, names_test, steering_valid, steering_test, inverse_valid, inverse_test=\
train_test_split(names_valid, steering_valid, inverse_valid, test_size=0.5)

In [8]:
train_gen=generator(names_train, steering_train, batch_size=128)
valid_gen=generator(names_valid, steering_valid, batch_size=128)
test_gen=generator (names_test, steering_test, batch_size=128)
    

In [9]:
grid=[{'get_model':modified_lenet, 'correction':0.09, \
       'preprocessing':lambda x:x,'optimizer':'adam','gpu':'0','name':'lenet_adam'},\
      {'get_model':modified_vgg, 'correction':0.09, \
       'preprocessing':lambda x:x,'optimizer':'adam','gpu':'0','name':'mod_adam_adam'},\
     {'get_model':nvidia_net, 'correction':0.09, \
       'preprocessing':preprocessing_yuv,'optimizer':'adam','gpu':'0','name':'nvidia_net_02_adam'},
      {'get_model':inception, 'correction':0.09, \
       'preprocessing':lambda x:x,'optimizer':'adam','gpu':'0','name':'inception_02_adam'},
     {'get_model':vgg16, 'correction':0.09, \
       'preprocessing':lambda x:x,'optimizer':'adam','gpu':'0','name':'vgg16_02_adam'},
      ]

In [10]:
import os
import pickle

def train_model(get_model, preprocessing, name, optimizer, gpu):
    import os
    print ('training', name)
    os.environ['CUDA_VISIBLE_DEVICES']=str(gpu)
    
    model=get_model()
    model.compile(loss='mse', optimizer=optimizer, metrics=['mean_squared_error'])
    
    if not os.path.exists('results/logs_%s'%name):
        os.mkdir('results/logs_%s'%name)
    
    
    checkpoint = ModelCheckpoint("results/model_%s.h5" % name, monitor='val_mean_squared_error', verbose=1,
                              save_best_only=True, mode='min')
    early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                               min_delta=0.001, patience=3,
                                verbose=False, mode='min')

    history = History()

    tb=TensorBoard(log_dir='results/logs_%s'%name, histogram_freq=0, write_graph=True, write_images=False)
    
    #generate_image_batch_tr(X_tr_names, y_tr, 64)
    train_gen=generate_image_batch_tr(names_train, steering_train, batch_size=128)
    valid_gen=generate_image_batch_tr(names_valid, steering_valid, batch_size=128)
    test_gen= generate_image_batch_tr (names_test, steering_test, batch_size=128)
    

    
    ##todo - nb_epoch=100
    q=model.fit_generator(train_gen, \
                    samples_per_epoch=len(names_train),\
                    nb_epoch=100,\
                    verbose=1,\
                    validation_data=valid_gen, \
                    nb_val_samples=len(names_valid),\
                    callbacks=[checkpoint, early_stop,history,tb]\
                   )
    
    f=open('results/history_%s.pk1' % name,'wb')
    pickle.dump(history.history,f,-1)
    f.close()
    
    model.load_weights("results/model_%s.h5" % name)
    
    res=model.evaluate_generator(test_gen,val_samples=len(names_test))[0]
    return q,res

In [ ]:
res=[train_model(**a) for a in grid[1:]]

In [35]:
def get_small_model():
    # ----------------------
    # Model - ideas from VG type network
    inp = Input(shape=(40,160,3))
    # First convolution is for model to determine the 'best' colorspace weights
    x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(inp)
    # Reduce dimensions
    x = MaxPooling2D((2,2))(x) #20x80

    # First convolution la yer
    x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
    x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
    x1 = MaxPooling2D((2,2))(x1) #10x40
    x1 = Dropout(0.5)(x1)
    flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

    # Second convolution layer
    x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
    x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
    x2 = MaxPooling2D((2,2))(x2) #5x20
    x2 = Dropout(0.5)(x2)
    flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

    # Second convolution layer
    x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
    x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
    x3 = MaxPooling2D((2,2))(x3) #2x10
    x3 = Dropout(0.5)(x3)
    flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

    # Merge the flattened ouputs after each convolution layer
    x4 = merge([flat1, flat2, flat3], mode='concat')
    # Fully connected layers
    x5 = Dense(512, activation='relu')(x4)
    x6 = Dense(128, activation='relu')(x5)
    x7 = Dense(16, activation='relu')(x6)
    out = Dense(1, activation='linear')(x7)

    model = Model(input=inp, output=out)
    #model.summary()
    return model





In [ ]:
q=train_model(get_small_model, proc_img, 'small_model', Adam(lr=0.0001), '1')

In [30]:
%%sh

mkdir results

In [69]:
model=modified_vgg()

In [70]:
checkpoint = ModelCheckpoint('lap1.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_mean_squared_error', verbose=1,
                              save_best_only=False, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                               min_delta=0.001, patience=3,
                                verbose=1, mode='min')



In [71]:
lap1=[i for i in range(len(names_valid)) if 'map1' in names_valid[i]]

In [72]:
valid_gen_lap1=generator (names_valid[lap1], steering_valid[lap1], batch_size=128)



In [ ]:
model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['mean_squared_error'])
    

q=model.fit_generator(train_gen, \
                    samples_per_epoch=len(names_train),\
                    nb_epoch=100,\
                    verbose=1,\
                    validation_data=valid_gen, \
                    nb_val_samples=len(names_valid),\
                    callbacks=[checkpoint, early_stop]\
                   )

In [45]:
def modified_vgg():
    #got and a bit modified from https://github.com/Valtgun/ 
    # ----------------------
    # Model - ideas from VG type network
    inp = Input(shape=(160,320,3))


    x=Cropping2D(cropping=((60,20), (0,0)))(inp)

    x=Lambda(lambda x: (x / 255.0) - 0.5)(x)

    x= MaxPooling2D((2,2))(x)
    # First convolution is for model to determine the 'best' colorspace weights
    x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(x)
    # Reduce dimensions
    x = MaxPooling2D((2,2))(x) #20x80

    # First convolution layer
    x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
    x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
    x1 = MaxPooling2D((2,2))(x1) #10x40
    x1 = Dropout(0.5)(x1)
    flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

    # Second convolution layer
    x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
    x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
    x2 = MaxPooling2D((2,2))(x2) #5x20
    x2 = Dropout(0.5)(x2)
    flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

    # Second convolution layer
    x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
    x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
    x3 = MaxPooling2D((2,2))(x3) #2x10
    x3 = Dropout(0.5)(x3)
    flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

    # Merge the flattened ouputs after each convolution layer
    x4 = merge([flat1, flat2, flat3], mode='concat')
    # Fully connected layers
    x5 = Dense(512, activation='relu')(x4)
    x6 = Dense(128, activation='relu')(x5)
    x7 = Dense(16, activation='relu')(x6)
    out = Dense(1, activation='linear')(x7)

    model = Model(input=inp, output=out)
    #model.summary()
    return model

In [86]:
def get_images(names, y, batch_size = 32, preprocessing = lambda x:x, inverse=None):
    images = []
    angles = np.zeros((len(names)),dtype=float)
    for now in range(len(names)):

        image = cv2.imread(names[now])
        angle=y[now]
        
        images.append(preprocessing(image))
        angles[now] = y[now]

            
        
    return np.array(images), angles

In [87]:
test_set=get_images(names_test, steering_test)

In [54]:
len(test_set[0])

NameError: name 'test_set' is not defined

In [111]:
model=nvidia_net()
model.load_weights('~/Downloads/model_nvidia_net_02_adam.h5')
preds=model.predict(test_set[0])

model=modified_vgg()
model.load_weights('~/Downloads/model_mod_adam_adam.h5')
pred_vgg=model.predict(test_set[0])



In [112]:
model.load_weights('results/model_nvidia_net_02_adam.h5')

In [100]:
model_yuv=np.array([preprocessing_yuv(a) for a in test_set[0]])

In [113]:
preds=model.predict(test_set[0])

In [105]:
model=modified_vgg()

model.load_weights('results/model_mod_adam_adam.h5')

pred_vgg=model.predict(test_set[0])

In [114]:
from sklearn.metrics import mean_squared_error

mean_squared_error(pred_vgg,test_set[1]), mean_squared_error(preds,test_set[1]), mean_squared_error(preds,pred_vgg)

(0.01519098718431436, 0.018835543697404675, 0.011245054)

import pickle

f=open('names.pk1','wb')
pickle.dump([names_test, names_train, names_valid, steering_test, steering_train, steering_valid], f, -1)
f.close()

In [9]:
# ----------------------
# Model - ideas from VG type network
inp = Input(shape=(40,160,3))
# First convolution is for model to determine the 'best' colorspace weights
x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(inp)
# Reduce dimensions
x = MaxPooling2D((2,2))(x) #20x80

# First convolution layer
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2))(x1) #10x40
x1 = Dropout(0.5)(x1)
flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

# Second convolution layer
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x2 = MaxPooling2D((2,2))(x2) #5x20
x2 = Dropout(0.5)(x2)
flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

# Second convolution layer
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
x3 = MaxPooling2D((2,2))(x3) #2x10
x3 = Dropout(0.5)(x3)
flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

# Merge the flattened ouputs after each convolution layer
x4 = merge([flat1, flat2, flat3], mode='concat')
# Fully connected layers
x5 = Dense(512, activation='relu')(x4)
x6 = Dense(128, activation='relu')(x5)
x7 = Dense(16, activation='relu')(x6)
out = Dense(1, activation='linear')(x7)

model = Model(input=inp, output=out)
model.summary()




____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 40, 160, 3)    0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 40, 160, 3)    12          input_1[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 20, 80, 3)     0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 20, 80, 32)    896         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [10]:
def generate_image_batch_tr(names, y_data, batch_size = 32):
    total_items = len(names)
    curr_item = 0
    while (True):
        image_data = np.zeros((batch_size,40,160, 3),dtype=float)
        steering_data = np.zeros((batch_size),dtype=float)
        for j in range(batch_size):
            image_name = names[curr_item]
            image = cv2.imread(image_name)
            image_data[j] = proc_img(image)
            steering_data[j] = y_data[curr_item]
            curr_item = (curr_item+1)%total_items
        yield image_data, steering_data

In [11]:
# ----------------------
# Model - ideas from VG type network
inp = Input(shape=(40,160,3))
# First convolution is for model to determine the 'best' colorspace weights
x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(inp)
# Reduce dimensions
x = MaxPooling2D((2,2))(x) #20x80

# First convolution la yer
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2))(x1) #10x40
x1 = Dropout(0.5)(x1)
flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

# Second convolution layer
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x2 = MaxPooling2D((2,2))(x2) #5x20
x2 = Dropout(0.5)(x2)
flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

# Second convolution layer
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
x3 = MaxPooling2D((2,2))(x3) #2x10
x3 = Dropout(0.5)(x3)
flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

# Merge the flattened ouputs after each convolution layer
x4 = merge([flat1, flat2, flat3], mode='concat')
# Fully connected layers
x5 = Dense(512, activation='relu')(x4)
x6 = Dense(128, activation='relu')(x5)
x7 = Dense(16, activation='relu')(x6)
out = Dense(1, activation='linear')(x7)

model = Model(input=inp, output=out)
model.summary()




____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 40, 160, 3)    0                                            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 40, 160, 3)    12          input_2[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 20, 80, 3)     0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 20, 80, 32)    896         maxpooling2d_5[0][0]             
___________________________________________________________________________________________

In [ ]:
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("model_test_lenet5.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=0, mode='min')


model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['mean_squared_error'])


print ('Split data')
image_names_full, y_data_full = images.img.values, images.real.values



print ('Start training')
# Training and validation inputs are fed from generators
# Number of samples based on data_set size and adjusted to fit batch size
history2 = model.fit_generator(generate_image_batch_tr(names_train, steering_train, 64),
                               samples_per_epoch=len(steering_train),
                              nb_epoch=100,
                              validation_data=generate_image_batch_tr(names_valid, steering_valid, 32),
                              nb_val_samples=len(steering_valid),  callbacks=[checkpoint, early_stop])

Split data
Start training
Epoch 1/100
270144/270103 [==============================] - 261s - loss: 0.0062 - mean_squared_error: 0.0062 - val_loss: 0.0050 - val_mean_squared_error: 0.0050
Epoch 23/100
105088/270103 [==========>...................] - ETA: 136s - loss: 0.0061 - mean_squared_error: 0.0061

In [ ]:
model.evaluate_generator(generate_image_batch_tr(X_test_names, y_test, 64), len(y_test))

In [21]:
train_gen=generator(names_train, steering_train, batch_size=128, preprocessing=proc_img)
valid_gen=generator(names_valid, steering_valid, batch_size=128, preprocessing=proc_img)
test_gen=generator (names_test, steering_test, batch_size=128, preprocessing=proc_img)

In [23]:
# ----------------------
# Model - ideas from VG type network
inp = Input(shape=(40,160,3))
# First convolution is for model to determine the 'best' colorspace weights
x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(inp)
# Reduce dimensions
x = MaxPooling2D((2,2))(x) #20x80

# First convolution la yer
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2))(x1) #10x40
x1 = Dropout(0.5)(x1)
flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

# Second convolution layer
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x2 = MaxPooling2D((2,2))(x2) #5x20
x2 = Dropout(0.5)(x2)
flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

# Second convolution layer
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
x3 = MaxPooling2D((2,2))(x3) #2x10
x3 = Dropout(0.5)(x3)
flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

# Merge the flattened ouputs after each convolution layer
x4 = merge([flat1, flat2, flat3], mode='concat')
# Fully connected layers
x5 = Dense(512, activation='relu')(x4)
x6 = Dense(128, activation='relu')(x5)
x7 = Dense(16, activation='relu')(x6)
out = Dense(1, activation='linear')(x7)

model = Model(input=inp, output=out)
model.summary()





____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 40, 160, 3)    0                                            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 40, 160, 3)    12          input_2[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 20, 80, 3)     0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 20, 80, 32)    896         maxpooling2d_5[0][0]             
___________________________________________________________________________________________

In [ ]:
checkpoint = ModelCheckpoint("model_test_lenet4_gen.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=0, mode='min')

# Compile, train and save
#model.compile(optimizer=Adam(lr=FLAGS.learn_rate), loss='mse')


model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['mean_squared_error'])


print ('Split data')
image_names_full, y_data_full = images.img.values, images.real.values



print ('Start training')
# Training and validation inputs are fed from generators
# Number of samples based on data_set size and adjusted to fit batch size
history = model.fit_generator(train_gen, \
                              samples_per_epoch=len(names_train),\
                              nb_epoch=5,
                              validation_data=generate_image_batch_tr(X_val_names, y_val, 32),
                              nb_val_samples=len(names_valid),  callbacks=[checkpoint, early_stop])

In [26]:
history.history

{'loss': [0.060880179450486682,
  0.049645218945222405,
  0.042601410672501012,
  0.036687453014173969,
  0.031932293409854609],
 'mean_squared_error': [0.060880179439676205,
  0.049645219045759838,
  0.042601410627097012,
  0.036687453050929589,
  0.03193229340444937],
 'val_loss': [0.051935428072830227,
  0.043243855862443933,
  0.036552703293286734,
  0.030200143396560526,
  0.025791173892661827],
 'val_mean_squared_error': [0.051935428072830227,
  0.043243855862443933,
  0.036552703293286734,
  0.030200143396560526,
  0.025791173892661827]}